# Multivariate polynomials implementations

**Contributed by**: Benoît Legat

The SumOfSquares package is built on top of the [MultivariatePolynomials](https://github.com/JuliaAlgebra/MultivariatePolynomials.jl)
abstract interface. [DynamicPolynomials](https://github.com/JuliaAlgebra/DynamicPolynomials.jl/)
is an implementation of this abstract interface so it can be used with
SumOfSquares. Moreover, any other implementation can be used as well. To
illustrate, we solve of [Blekherman2012; Examples 3.38](@cite) with
[TypedPolynomials](https://github.com/JuliaAlgebra/TypedPolynomials.jl),
another implementation of [MultivariatePolynomials](https://github.com/JuliaAlgebra/MultivariatePolynomials.jl).

In [1]:
import TypedPolynomials
TypedPolynomials.@polyvar x y
using SumOfSquares
import CSDP
model = SOSModel(CSDP.Optimizer)
con_ref = @constraint(model, 2x^4 + 5y^4 - x^2*y^2 >= -2(x^3*y + x + 1))
optimize!(model)
solution_summary(model)

CSDP 6.2.0
This is a pure primal feasibility problem.
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 Ap: 7.39e-01 Pobj:  0.0000000e+00 Ad: 1.00e+00 Dobj:  9.2771541e+01 
Iter:  2 Ap: 1.00e+00 Pobj:  0.0000000e+00 Ad: 1.00e+00 Dobj:  5.3405053e+01 


* Solver : CSDP

* Status
  Result count       : 1
  Termination status : OPTIMAL
  Message from the solver:
  "Problem solved to optimality."

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : FEASIBLE_POINT
  Objective value    : 0.00000e+00
  Dual objective value : 0.00000e+00

* Work counters
  Solve time (sec)   : 4.36068e-04


We see that the problem is feasible. The Sum-of-Squares decomposition can be
obtained as follows:

In [2]:
sos_decomposition(con_ref)

(-0.6797543969221541 - 0.09754507709792246*y - 0.4175950563713448*x + 2.1931518901396174*y^2 - 0.1269606244678339*x*y - 0.8025867547845221*x^2)^2 + (0.9043573827480671 - 0.6455003192597278*y + 0.2742159536999832*x - 0.10770174782358159*y^2 - 1.2293138068800717*x*y - 0.9300160939847*x^2)^2 + (-0.5739168240154698 - 1.38535050221548*y - 0.6219655378569912*x - 0.2040924100740057*y^2 - 0.15182215880489783*x*y + 0.4443832632766482*x^2)^2 + (0.5703021360913615 - 0.5184093413621602*y + 0.3436728787624473*x + 0.2555950605470665*y^2 + 0.7618441294650047*x*y - 0.02090567376761348*x^2)^2 + (0.041400837217767876 + 0.04911040272637946*y - 0.49990658891857337*x - 0.24792145939081137*y^2 + 0.2972445689917418*x*y - 0.5054184235227418*x^2)^2 + (-0.2524531788141328 - 0.06336647012459033*y + 0.253942572224051*x - 0.10018744153048117*y^2 + 0.059601795985895156*x*y - 0.19381241255811912*x^2)^2

Why is there several implementations ?
Depending in the use-case, one implementation may be more appropriate than
another one. [TypedPolynomials](https://github.com/JuliaAlgebra/TypedPolynomials.jl)
is faster than [DynamicPolynomials](https://github.com/JuliaAlgebra/DynamicPolynomials.jl/)
but it requires new compilation whenever the list of variables changes.
This means that [TypedPolynomials](https://github.com/JuliaAlgebra/TypedPolynomials.jl)
is not appropriate when the number of variables is dynamic or too large.
However, for a small number of variables, it can be faster.
When solving Sum-of-Squares programs, the time is mostly taken by the Semidefinite programming solver.
The time taken by SumOfSquares/JuMP/MathOptInterface are usually negligible
or it time is taken by manipulation of JuMP or MathOptInterface functions
therefore using TypedPolynomials over DynamicPolynomials may not make much difference in most cases.

One case for which using TypedPolynomials might be adequate is when
using domain defined by equalities (possibly also with inequalities).
Indeed, in that case, SumOfSquares computes the corresponding Gröbner basis which
may take a non-negligible amount of time for large systems of equalities.

To illustrate this, consider the computation of Gröbner basis for the
following system from [CLO05, p. 17].
The time taken by TypedPolynomials is below:

[CLO05] Cox, A. David & Little, John & O'Shea, Donal
*Using Algebraic Geometry*.
Graduate Texts in Mathematics, **2005**.
https://doi.org/10.1007/b138611

In [3]:
using BenchmarkTools
@btime let
    TypedPolynomials.@polyvar x y
    S = @set x^3 * y + x == 2x^2 * y^2 && 3x^4 == y
    SemialgebraicSets.compute_gröbner_basis!(S.I)
end

  293.347 μs (6275 allocations: 1.33 MiB)


true

The time taken by DynamicPolynomials is as follows:

In [4]:
import DynamicPolynomials
@btime let
    DynamicPolynomials.@polyvar x y
    S = @set x^3 * y + x == 2x^2 * y^2 && 3x^4 == y
    SemialgebraicSets.compute_gröbner_basis!(S.I)
end

  419.852 μs (8486 allocations: 1.21 MiB)


true

We see that TypedPolynomials is faster.
The time is still negligible for this small system but for larger systems, choosing TypedPolynomials may be helpful.
We can use this system in a Sum-of-Squares constraint as follows:

In [5]:
TypedPolynomials.@polyvar x y
S = @set x^3 * y + x == 2x^2 * y^2 && 3x^4 == y
poly = -6x - 4y^3 + 2x*y^2 + 6x^3 - 3y^4 + 13x^2 * y^2
model = Model(CSDP.Optimizer)
con_ref = @constraint(model, poly in SOSCone(), domain = S)
optimize!(model)
solution_summary(model)

Success: SDP solved
Primal objective value: 0.0000000e+00 
Dual objective value: 0.0000000e+00 
Relative primal infeasibility: 2.23e-16 
Relative dual infeasibility: 5.00e-11 
Real Relative Gap: 0.00e+00 
XZ Relative Gap: 3.04e-10 
DIMACS error measures: 2.78e-16 0.00e+00 5.00e-11 0.00e+00 0.00e+00 3.04e-10
CSDP 6.2.0
This is a pure primal feasibility problem.
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 Ap: 7.64e-01 Pobj:  0.0000000e+00 Ad: 8.07e-01 Dobj:  2.6104524e+00 
Iter:  2 Ap: 7.71e-01 Pobj:  0.0000000e+00 Ad: 7.40e-01 Dobj: -5.9089088e-02 
Iter:  3 Ap: 8.08e-01 Pobj:  0.0000000e+00 Ad: 7.65e-01 Dobj:  5.5548423e-01 
Iter:  4 Ap: 7.56e-01 Pobj:  0.0000000e+00 Ad: 7.79e-01 Dobj:  4.8507209e-01 
Iter:  5 Ap: 6.95e-01 Pobj:  0.0000000e+00 Ad: 6.77e-01 Dobj:  4.3029792e-01 
Iter:  6 Ap: 6.35e-01 Pobj:  0.0000000e+00 Ad: 6.83e-01 Dobj:  2.2138686e-01 
Iter:  7 Ap: 6.21e-01 Pobj:  0.0000000e+00 Ad: 6.43e-01 Dobj:  1.2987235e-01 
Iter:  8 Ap: 

* Solver : CSDP

* Status
  Result count       : 1
  Termination status : OPTIMAL
  Message from the solver:
  "Problem solved to optimality."

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : FEASIBLE_POINT
  Objective value    : 0.00000e+00
  Dual objective value : 0.00000e+00

* Work counters
  Solve time (sec)   : 2.43592e-03


We obtain the following decomposition:

In [6]:
dec = sos_decomposition(con_ref, 1e-6)

(-5.120614652559952e-5 + 0.0010584845447259056*y + 0.000340808664933532*x + 8.157919816740142*y^2 - 32.77710962574069*x*y + 29.436684176438884*x^2)^2 + (-1.0587801055835377e-5 + 0.00010602040011976172*y + 0.0010736461776558717*x + 26.098857671706984*y^2 - 16.09375738552047*x*y - 25.152942986047297*x^2)^2 + (2.970207075143178e-5 + 0.000794789039104227*y - 0.00016584049945627324*x - 0.12253485455459644*y^2 - 0.0918840250668785*x*y - 0.06835226465828073*x^2)^2 + (-5.17371414837382e-7 - 0.03721185223726042*y + 0.049797912979844754*x - 0.00016553966856285108*y^2 - 0.00012344309755984585*x*y - 9.081294614806447e-5*x^2)^2

We can verify that it is correct as follows:

In [7]:
rem(dec - poly, S.I)

3.616651653303066628549364440784429086672702169380499981343746185302734375e-09 + 7.939328636338026522517119874550896058852417925850321085025102664263115294040558e-08y + 5.937935913909069103493283574397747333233173076923076923076923076923076923076939e-08x - 8.92394073232828134722982582616168656386435031890869140625e-06y² - 1.3343841933410004185756037031751475296914577484130859375e-05xy - 4.9814204763931092811990453128601075150072574615478515625e-06x² - 6.626560426307529875581773618857065836588541666666666666666666666666671272623229e-08y³ - 7.115966675167821146411976466576258341471354166666666666666666666666664363688385e-08xy² - 5.8665363968814893645686847634124205796979367733001708984375e-08x²y + 4.281866075381493339171776404747596153846153846153846153846153846153846153846431e-09y⁴

Note that the difference between `dec` and `poly` is larger
than between the full gram matrix because `dec` is obtained by dropping
the lowest eigenvalues with the threshold `1e-6`; see `sos_decomposition`.

In [8]:
rem(gram_matrix(con_ref) - poly, S.I)

4.28217951213599115010063358120750987012570476508699357509613037109375e-09 + 2.410877415089833983113631389640975391021362529733242132724859775643865390399207e-09y + 1.900553150755093576243290534386268028846153846153846153846153846154529075482902e-09x + 4.282181762828296545109196813427843153476715087890625e-09y² + 1.317522480004384988205856643617153167724609375e-15xy + 4.28216934307557028915880437125451862812042236328125e-09x² - 4.352074256530613638460636138916015625e-14y³ + 6.128431095930864103138446807861328125e-14xy² + 1.8955670366693766482057981193065643310546875e-13x²y + 4.455098580645146564795420720027043269230769230769230769230769230763767396138843e-09y⁴

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*